## Imports

In [3]:
import pandas as pd

from pathlib import Path
from tqdm import tqdm

## Params

In [4]:
EQUITY_TYPE = 'despac'

## Load Data

In [6]:
partition_dir = Path(f"../data/option_data/{EQUITY_TYPE}/partition")

dfs = []

for file in tqdm(list(partition_dir.glob("*.parquet")), desc="Reading parquet files"):
    file_date = pd.to_datetime(file.stem, format="%Y-%m-%d")

    df = pd.read_parquet(file)
    df["file_date"] = file_date
    dfs.append(df)

options_data = pd.concat(dfs, ignore_index=True)

Reading parquet files: 100%|██████████| 1730/1730 [00:23<00:00, 74.34it/s]


## Clean Data

In [7]:
# Convert columns to appropriate types
print("\nConverting date columns...")
options_data['date'] = pd.to_datetime(options_data['date'], errors='coerce')
options_data['file_date'] = pd.to_datetime(options_data['file_date'], errors='coerce')

# String columns
print("Converting string columns...")
string_cols = ['ticker', 'contractID', 'symbol', 'type', 'expiration_date']
for col in tqdm(string_cols, desc="  String columns"):
    if col in options_data.columns:
        options_data[col] = options_data[col].astype(str).replace('nan', '').replace('None', '')

# Datetime columns (expiration might be in string format)
print("Converting expiration datetime...")
if 'expiration' in options_data.columns:
    options_data['expiration'] = pd.to_datetime(options_data['expiration'], errors='coerce')

# Float columns (financial metrics)
print("Converting float columns...")
float_cols = ['strike', 'last', 'mark', 'bid', 'ask', 'implied_volatility', 
              'delta', 'gamma', 'theta', 'vega', 'rho']
for col in tqdm(float_cols, desc="  Float columns"):
    if col in options_data.columns:
        options_data[col] = pd.to_numeric(options_data[col], errors='coerce')

# Integer columns (can have NaN)
print("Converting integer columns...")
int_cols = ['bid_size', 'ask_size', 'volume', 'open_interest']
for col in tqdm(int_cols, desc="  Integer columns"):
    if col in options_data.columns:
        options_data[col] = pd.to_numeric(options_data[col], errors='coerce').astype('Int64')  # Nullable integer

options_data = options_data.drop(columns=['expiration_date', 'no_options_data', 'all_tickers_skipped', 'file_date'], errors='ignore')

# Clean data
options_data = options_data.drop(columns=['symbol'])
options_data = options_data.rename(columns={'contractID': 'id', 'type': 'option_type'})

options_data["option_type"] = options_data["option_type"].replace({
    "call": "C",
    "put": "P",
})

options_data = options_data.set_index(['date', 'ticker', 'expiration', 'strike', 'option_type', 'id']).sort_index()
options_data = options_data.dropna()

options_data['days_till_expiration'] = (options_data.index.get_level_values('expiration') - options_data.index.get_level_values('date')).days


options_data.to_parquet(f"../data/option_data/{EQUITY_TYPE}/clean_options_data.parquet", engine="pyarrow")


Converting date columns...
Converting string columns...


  String columns: 100%|██████████| 5/5 [00:16<00:00,  3.32s/it]


Converting expiration datetime...
Converting float columns...


  Float columns: 100%|██████████| 11/11 [01:09<00:00,  6.36s/it]


Converting integer columns...


  Integer columns: 100%|██████████| 4/4 [00:46<00:00, 11.61s/it]


In [14]:
import os 
import pandas as pd
EQUITY_TYPE='despac'
if os.path.exists(f"../data/option_data/{EQUITY_TYPE}/clean_options_data.parquet"):
    data = pd.read_parquet(f"../data/option_data/{EQUITY_TYPE}/clean_options_data.parquet", engine="pyarrow")
    pd.concat([data.head(1), data.tail(1)])

In [15]:
len(data)

35122686

In [16]:
display(data.head())
display(data.tail())

last  \
date       ticker expiration strike option_type id                         
2019-10-28 OSW    2019-11-15 12.5   C           OSW191115C00012500   0.0   
                                    P           OSW191115P00012500   0.0   
                             15.0   C           OSW191115C00015000   0.0   
                                    P           OSW191115P00015000   0.0   
                             17.5   C           OSW191115C00017500   0.0   

                                                                    mark  \
date       ticker expiration strike option_type id                         
2019-10-28 OSW    2019-11-15 12.5   C           OSW191115C00012500  4.08   
                                    P           OSW191115P00012500  0.01   
                             15.0   C           OSW191115C00015000  1.98   
                                    P           OSW191115P00015000  0.01   
                             17.5   C           OSW191115C00017500  0.01   

                                                                     bid  \
date       ticker expiration strike option_type id                         
2019-10-28 OSW    2019-11-15 12.5   C           OSW191115C00012500  2.75   
                                    P           OSW191115P00012500  0.00   
                             15.0   C           OSW191115C00015000  1.10   
                                    P           OSW191115P00015000  0.00   
                             17.5   C           OSW191115C00017500  0.00   

                                                                    bid_size  \
date       ticker expiration strike option_type id                             
2019-10-28 OSW    2019-11-15 12.5   C           OSW191115C00012500        60   
                                    P           OSW191115P00012500         0   
                             15.0   C           OSW191115C00015000        65   
                                    P           OSW191115P00015000         0   
                             17.5   C           OSW191115C00017500         0   

                                                                     ask  \
date       ticker expiration strike option_type id                         
2019-10-28 OSW    2019-11-15 12.5   C           OSW191115C00012500  5.40   
                                    P           OSW191115P00012500  0.80   
                             15.0   C           OSW191115C00015000  2.85   
                                    P           OSW191115P00015000  0.65   
                             17.5   C           OSW191115C00017500  0.75   

                                                                    ask_size  \
date       ticker expiration strike option_type id                             
2019-10-28 OSW    2019-11-15 12.5   C           OSW191115C00012500         1   
                                    P           OSW191115P00012500        52   
                             15.0   C           OSW191115C00015000         6   
                                    P           OSW191115P00015000        35   
                             17.5   C           OSW191115C00017500        35   

                                                                    volume  \
date       ticker expiration strike option_type id                           
2019-10-28 OSW    2019-11-15 12.5   C           OSW191115C00012500       0   
                                    P           OSW191115P00012500       0   
                             15.0   C           OSW191115C00015000       0   
                                    P           OSW191115P00015000       0   
                             17.5   C           OSW191115C00017500       0   

                                                                    open_interest  \
date       ticker expiration strike option_type id                                  
2019-10-28 OSW    2019-11-15 12.5   C           OSW191115C00012500              0   
            

last  \
date       ticker expiration strike option_type id                          
2025-12-29 ZURA   2026-06-18 2.5    P           ZURA260618P00002500  0.25   
                             5.0    C           ZURA260618C00005000  2.44   
                                    P           ZURA260618P00005000  0.00   
                             7.5    C           ZURA260618C00007500  1.21   
                                    P           ZURA260618P00007500  0.00   

                                                                     mark  \
date       ticker expiration strike option_type id                          
2025-12-29 ZURA   2026-06-18 2.5    P           ZURA260618P00002500  0.01   
                             5.0    C           ZURA260618C00005000  2.48   
                                    P           ZURA260618P00005000  0.01   
                             7.5    C           ZURA260618C00007500  2.70   
                                    P           ZURA260618P00007500  3.20   

                                                                      bid  \
date       ticker expiration strike option_type id                          
2025-12-29 ZURA   2026-06-18 2.5    P           ZURA260618P00002500  0.00   
                             5.0    C           ZURA260618C00005000  0.05   
                                    P           ZURA260618P00005000  0.00   
                             7.5    C           ZURA260618C00007500  0.50   
                                    P           ZURA260618P00007500  0.90   

                                                                     bid_size  \
date       ticker expiration strike option_type id                              
2025-12-29 ZURA   2026-06-18 2.5    P           ZURA260618P00002500         0   
                             5.0    C           ZURA260618C00005000        16   
                                    P           ZURA260618P00005000         0   
                             7.5    C           ZURA260618C00007500         1   
                                    P           ZURA260618P00007500        47   

                                                                     ask  \
date       ticker expiration strike option_type id                         
2025-12-29 ZURA   2026-06-18 2.5    P           ZURA260618P00002500  0.5   
                             5.0    C           ZURA260618C00005000  4.9   
                                    P           ZURA260618P00005000  4.9   
                             7.5    C           ZURA260618C00007500  4.9   
                                    P           ZURA260618P00007500  5.5   

                                                                     ask_size  \
date       ticker expiration strike option_type id                              
2025-12-29 ZURA   2026-06-18 2.5    P           ZURA260618P00002500         7   
                             5.0    C           ZURA260618C00005000       116   
                                    P           ZURA260618P00005000       119   
                             7.5    C           ZURA260618C00007500        92   
                                    P           ZURA260618P00007500        15   

                                                                     volume  \
date       ticker expiration strike option_type id                            
2025-12-29 ZURA   2026-06-18 2.5    P           ZURA260618P00002500      20   
                             5.0    C           ZURA260618C00005000       0   
                                    P           ZURA260618P00005000       0   
                             7.5    C           ZURA260618C00007500       0   
                                    P           ZURA260618P00007500       0   

                                                                     open_interest  \
date       ticker expiration strike option_type id                                   
2025-12-29 ZURA   2026-06-18 2.5    P           ZURA26